# Performance measure
- 만들어진 모델의 성능은 어떻게 평가할 것인가?

## Regression metrics

### Mean Absolute Error

> $MAE = \frac{1}{n}\displaystyle\sum_{i=1}^{m}{|y_i - \hat{y_i}|} = \displaystyle\sum_{i=1}^{m}{|e_i|}$
- 잔차의 절댓값의 Sum

In [1]:
from sklearn.metrics import mean_absolute_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0, 2, 8]
mean_absolute_error(y_true, y_pred)

0.5

### Root Mean Squared Error (RMSE)

> $RMSE = \sqrt{\frac{1}{n}\displaystyle\sum_{i=1}^{m}{(y_i - \hat{y_i})^2}}$
- 잔차 제곱의 Sum의 루트
- RMSE는 `sklearn`에서 제공되지 않기 때문에 MSE를 구해서 직접 계산해야 함

In [2]:
from sklearn.metrics import mean_squared_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0, 2, 8]
mean_squared_error(y_true, y_pred)

0.375

### R squared

> $R^2 = 1 - \frac{\sum_{i}{(y_i-\hat{y}_i)^2}}{\sum_{i}{(y_i-\mu)^2}}$
- 0과 1 사이의 숫자
- 크면 클수록 높은 적합도를 지님

In [4]:
from sklearn.metrics import r2_score
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0, 2, 8]
r2_score(y_true, y_pred)

0.9486081370449679

## Training & Test data set
- 모델을 만드는 데이터과 평가받는 데이터를 나누는 것. 

- Training한 데이터로 다시 Test를 할 경우, Training 데이터에 과도하게 fitting(:=overfitting)된 모델이 사용될 수 있음
- 새로운 데이터가 출현했을 대, 기존 모델과의 차이 존재
- 모델은 새로운 데이터를 처리할 수 있도록 generalize 되어야 함
- 이를 위해 Training Set과 Test Set을 분리함

### Holdout Method (Sampling)
- 데이터를 Training과 Test로 나누어서 모델을 생성하고 테스트하는 기법
- 가장 일반적인 모델 생성을 위한 데이터 랜덤 샘플링 기법
- Training과 Test를 나누는 비율은 데이터의 크기에 따라 다름
- 일반적으로 Training Data 2/3, Test Data 1/3을 활용함

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

X, y = np.arange(10).reshape((5,2)), range(5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
print("X_train\n", X_train)
print("X_test\n", X_test)
print("y_train\n", y_train)
print("y_test\n", y_test)

X_train
 [[4 5]
 [0 1]
 [6 7]]
X_test
 [[2 3]
 [8 9]]
y_train
 [2, 0, 3]
y_test
 [1, 4]


---

# Linear Regression with Sklearn

## Dataset - Boston House Price Dataset
- 머신러닝 등 데이터 분석을 처음 배울 때 가장 대표적으로 사용하는 Example Dataset

## 데이터 로딩

In [14]:
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [19]:
boston = load_boston()
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [20]:
x_data = boston.data
y_data = boston.target.reshape(boston.target.size, 1)

x_data[:3], x_data.shape

(array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
         1.5300e+01, 3.9690e+02, 4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9690e+02, 9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9283e+02, 4.0300e+00]]),
 (506, 13))

## 데이터 스케일링

In [28]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler().fit(x_data)
# standard_scale = preprocessing.StandardScaler().fit(x_data)
x_scaled_data = minmax_scale.transform(x_data)

x_scaled_data[:3]

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, 0.00000000e+00,
        3.14814815e-01, 5.77505269e-01, 6.41606591e-01, 2.69203139e-01,
        0.00000000e+00, 2.08015267e-01, 2.87234043e-01, 1.00000000e+00,
        8.96799117e-02],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, 0.00000000e+00,
        1.72839506e-01, 5.47997701e-01, 7.82698249e-01, 3.48961980e-01,
        4.34782609e-02, 1.04961832e-01, 5.53191489e-01, 1.00000000e+00,
        2.04470199e-01],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, 0.00000000e+00,
        1.72839506e-01, 6.94385898e-01, 5.99382080e-01, 3.48961980e-01,
        4.34782609e-02, 1.04961832e-01, 5.53191489e-01, 9.89737254e-01,
        6.34657837e-02]])

## Train-Test Split

In [29]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size = 0.33)

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((339, 13), (167, 13), (339, 1), (167, 1))

## Linear Regression fitting

$y = w_1x_1 + w_2x_2 + w_3x_3 + \cdots + w_{13}x_{13} + w_0\cdot{1}$

In [31]:
from sklearn import linear_model

regr = linear_model.LinearRegression(fit_intercept = True,
                                     normalize = False,
                                     copy_X = True,
                                     n_jobs = 8)
regr.fit(X_train, y_train)
regr

LinearRegression(n_jobs=8, normalize=False)

In [33]:
# coefficient는 feature의 개수와 같이 13개
print('Coefficients : ', regr.coef_)
print('intercept : ', regr.intercept_)

Coefficients :  [[ -7.99271874   6.6647091    0.39235496   3.3176537   -9.99519066
   14.96370469   1.44066423 -18.06332196   7.62988635  -7.23227411
   -8.66169773   3.95431502 -20.91999637]]
intercept :  [29.26871536]


- 아래 두 코드는 같은 의미임

In [34]:
regr.predict(x_data[0].reshape(1, -1))

array([[-537.21060124]])

In [35]:
x_data[0].dot(regr.coef_.T) + regr.intercept_ # Xw와 같음

array([-537.21060124])

## Metric 측정

In [37]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_true = y_test
y_hat = regr.predict(X_test)

print('r2 score : ', r2_score(y_true, y_hat)) 
print('MAE : ', mean_absolute_error(y_true, y_hat))
print('MSE : ', mean_squared_error(y_true, y_hat))


r2 score :  0.7442122905628782
MAE :  3.382843744756578
MSE :  20.969623716102358
